# Running remotely in Vulkan Engine

Now that we've tested and run our policy locally, we can use Vulkan Engine to scale and serve it. \
The same code will run in the remote server, and it'll have the added benefits of scalability, monitoring and governance. 

This notebook will show you how to upload this policy to the server and run a backtest!

In [ ]:
import os

import pandas as pd
from vulkan.cli import client as vulkan
from vulkan.cli.context import Context
from vulkan.spec.dependency import INPUT_NODE, Dependency
from vulkan.spec.nodes import BranchNode, TerminateNode
from vulkan.spec.policy import PolicyDefinition

In [ ]:
ctx = Context()

## Creating the Policy

In [ ]:
policy_path = os.path.abspath("../examples/policies/local/")
print(f"Creating a Policy based on {policy_path}")

policy_id = vulkan.policy.create_policy(ctx, name="IsEven")

In [ ]:
def is_even_fn(input_node):
    if input_node["number"] % 2 == 0:
        return "even"
    return "odd"


branch_node_1 = BranchNode(
    name="branch_node_1",
    func=is_even_fn,
    choices=["even", "odd"],
    dependencies={INPUT_NODE: Dependency(INPUT_NODE)},
)

is_even = TerminateNode(
    name="is_even",
    return_status="Great",
    dependencies={"condition": Dependency("branch_node_1", "even")},
)

is_odd = TerminateNode(
    name="is_odd",
    return_status="Too_Bad",
    dependencies={"condition": Dependency("branch_node_1", "odd")},
)

is_even = PolicyDefinition(
    nodes=[
        branch_node_1,
        is_even,
        is_odd,
    ],
    input_schema={"number": "int"},
)

In [ ]:
policy_version = vulkan.policy_version.create(
    ctx,
    policy_id=policy_id,
    version_name="v1",
    spec=is_even.to_dict(),
    input_schema={"number": "int"},
)

## Running we can run our policy (both locally and remotely!)

In [ ]:
from numpy.random import randint

In [ ]:
run_id, ok = vulkan.run.trigger_run_by_policy_version_id(
    ctx, policy_version["policy_version_id"], {"number": randint(100)}
)
vulkan.run.get_run_data(ctx, run_id)